# Gym Exercise Recommendation

The goal of this project is to create a gym exercise recommendation algorithm.
It'll recommend exercises based on strength, type of exercise and equipment needed. 

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr


In [3]:
data = pd.read_csv(r'/Users/steelo/Library/CloudStorage/OneDrive-Personal/Dev/codeProjects/gym_recommendation/megaGymDataset.csv')

In [4]:
data.rename( columns={'Unnamed: 0':'ID'}, inplace=True )
data

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2913,2913,EZ-bar skullcrusher-,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,8.1,Average
2914,2914,Lying Close-Grip Barbell Triceps Press To Chin,NaN,Strength,Triceps,E-Z Curl Bar,Beginner,8.1,Average
2915,2915,EZ-Bar Skullcrusher - Gethin Variation,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN
2916,2916,TBS Skullcrusher,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN


In [5]:
sqldf("SELECT * FROM data GROUP BY Type", globals())

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,149,Roc Everest,None,Cardio,Abdominals,Other,Intermediate,NaN,None
1,1217,Kneeling Jump Squat,None,Olympic Weightlifting,Glutes,Barbell,Beginner,7.6,Average
2,96,Sledgehammer swing,The sledgehammer swing is an exercise working ...,Plyometrics,Abdominals,Other,Intermediate,8.6,Average
3,917,Bench Press With Short Bands,None,Powerlifting,Chest,Bands,Beginner,0.0,None
4,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,None
5,204,Stomach Vacuum,None,Stretching,Abdominals,Body Only,Beginner,8.5,Average
6,1159,Forward Drag with Press,None,Strongman,Chest,Other,Beginner,6.0,Average


In [ ]:
le = LabelEncoder() # Create a LabelEncoder object

# Fit the LabelEncoder to the data and transform it
data['Type_encoded'] = le.fit_transform(data['Type'])
data['Equipment_encoded'] = le.fit_transform(data['Equipment'])
data['Level_encoded'] = le.fit_transform(data['Level'])
data['BodyPart_encoded'] = le.fit_transform(data['BodyPart'])
# 
# # Inverse transform the encoded data to get the original text values
# data['Type_original'] = le.inverse_transform(data['Type_encoded'])
# data['Equipment_original'] = le.inverse_transform(data['Equipment_encoded'])
# data['Level_original'] = le.inverse_transform(data['Level_encoded'])
# data['BodyPart_original'] = le.inverse_transform(data['BodyPart_encoded'])

data

In [ ]:
heatmap_columns = ['Type_encoded', 'Equipment_encoded', 'BodyPart_encoded', 'Level_encoded']
sns.heatmap(data[heatmap_columns].corr(), annot=True, cmap='coolwarm')


In [13]:
# Compute Pearson correlation coefficient and p-value
correlation_coefficient, p_value = pearsonr(data['Type_encoded'], data['Level_encoded'])
print("Pearson correlation coefficient:", correlation_coefficient)
print("P-value:", p_value)

Pearson correlation coefficient: 0.1125080108593004
P-value: 1.0993840127818986e-09
